In [1]:
from subprocess import Popen, PIPE
import getpass
username='EZAGAYNAYA'
REALM='BEE.VIMPELCOM.RU'
password = getpass.getpass(prompt='Enter Your Ldap password:\n')
kinit = 'kinit'
kinit_args = [ kinit, '%s@%s' % (username, REALM) ]
kinit = Popen(kinit_args, stdin=PIPE, stdout=PIPE, stderr=PIPE)
kinit.stdin.write(bytearray(password + "\n", "ascii"))
kinit.stdin.flush()
kinit.wait()

Enter Your Ldap password:
 ········


0

In [2]:
from pyspark import SparkConf  # noqa: E402
from pyspark.sql import DataFrame as SparkDataFrame  # noqa: E402
from pyspark.sql import SparkSession  # noqa: E402
from pyspark.sql import functions as F  # noqa: E402
from pyspark.sql.functions import col, when, min, max, sum, mean, count, substring, countDistinct, last_day, concat_ws 
from pyspark.sql.types import StringType, LongType

SPARK_SETTINGS = [
    ('spark.yarn.queue', 'default'),
        ('spark.sql.sources.partitionOverwriteMode', 'dynamic'),
        ('hive.exec.dynamic.partition', 'true'),
        ('hive.exec.dynamic.partition.mode', 'nonstrict'),
        ('spark.default.parallelism', '400'),
        ('spark.driver.memory', '20G'),
        ('spark.driver.memoryOverhead', '2G'),
        ('spark.driver.cores', 4),
        ('spark.driver.maxResultSize', '150G'),
        ('spark.dynamicAllocation.enabled', 'false'),
        ('spark.dynamicAllocation.maxExecutors', 200),
        ('spark.executor.memory', '20G'),
        ('spark.executor.memoryOverhead', '4G'),
        ('spark.executor.cores', 6),
        ('spark.executor.instances', 10),
        ('spark.hadoop.mapreduce.input.fileinputformat.input.dir.recursive', 'true'),
        ('spark.hive.mapred.supports.subdirectories', 'true'),# драйвер (понадобится для чтения из RDBMS)
        ('spark.kryoserializer.buffer', '1000m'),
        ('spark.kryoserializer.buffer.max', '2000m'),
        ('spark.rpc.message.maxSize', 2000),
        ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
        ('spark.sql.execution.arrow.pyspark.enabled', 'true'),
        ('spark.sql.shuffle.partitions', '400'),
        ('spark.driver.extraClassPath', '/lib/oracle/19.8/client64/lib/ojdbc8.jar'), # драйвер (понадобится для чтения из RDBMS)
        ('spark.jars', '/lib/oracle/19.8/client64/lib/ojdbc8.jar'),
        ("spark.sql.hive.convertMetastoreOrc", " false"),
        ('spark.app.name', 'SVFomenkov_testing_pipeline')
]

app_name = 'ZEM_spark_session'

conf = SparkConf().setAppName(app_name).setMaster('yarn').setAll(SPARK_SETTINGS)
spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()
spark.sparkContext.setLogLevel('ERROR')
spark.conf.set("spark.sql.hive.convertMetastoreOrc", " false")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/31 11:47:31 WARN HiveConf: HiveConf of name hive.mapred.supports.subdirectories does not exist
25/10/31 11:47:39 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
from IPython.display import display, HTML
from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))
display(HTML("<style>div.jp-OutputArea-output pre {white-space: pre;}</style>"))

/tmp/ipykernel_5658/3383840084.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


#### Pandas

In [4]:
import pandas as pd
import numpy as np
import scipy as sp

import pickle
# import shap

import statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor 

import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection, datasets, linear_model, metrics

import seaborn as sns

import datetime as dt
from datetime import datetime
from datetime import date
import math

# TF-IDF:
import sklearn.feature_extraction.text
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve

from matplotlib.collections import LineCollection

import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['min', 'max', 'datetime', 'mean', 'sum']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


#### df Spark

In [5]:
# read traffic data:

table_name = 'beemetrics.DM_MOBILE_SUBSCRIBER_USAGE_DAILY'

# start_date = datetime.date(2025, 6, 1) # -- !!!
# end_date = datetime.date(2025, 9, 22) # -- !!!

# start_date = datetime.date(2025, 9, 22) # -- !!!
# end_date = datetime.date(2025, 10, 19) # -- !!!

start_date = datetime.date(2025, 9, 29) # -- !!!
end_date = datetime.date(2025, 10, 27) # -- !!!

df = (
   spark.read.table(table_name)
    .where(F.col('transaction_dt') >= start_date)
    .where(F.col('transaction_dt') <  end_date)
    .where(F.col('business_type_cd') == 'B2B')
    .where(~F.col('segment_cd').isin('GKA', 'GME', 'GLA', 'GSE', 'GSH'))
    # .where(F.col('subs_market_cd') == 'VIP')      # --!!!!
    # .where(F.col('account_type_cd').isin(['47', '120']))  # -- MK
    .groupby(
        F.col('transaction_dt'),
        F.col('subscriber_sk'),
        F.col('subscriber_num'),
        F.col('ban_num'),
        F.col('price_plan_cd'),
        F.col('segment_cd'),
        F.col('subs_market_cd'),
        # F.col('call_direction_cd'),
        # F.col('roaming_cd')
    )
    .agg(
        F.sum(F.when( (F.col('activity_cd') == 'VOICE') & (F.col('call_direction_cd') == 1) & (F.col('roaming_cd') == 'H'), F.col('usage_amt')/60 )).alias('voice_min_1_H'),
        F.sum(F.when( (F.col('activity_cd') == 'VOICE') & (F.col('call_direction_cd') == 2) & (F.col('roaming_cd') == 'H'), F.col('usage_amt')/60 )).alias('voice_min_2_H'),
        F.sum(F.when( (F.col('activity_cd') == 'VOICE') & (F.col('call_direction_cd') == 1) & (F.col('roaming_cd') != 'H'), F.col('usage_amt')/60 )).alias('voice_min_1_R'),
        F.sum(F.when( (F.col('activity_cd') == 'VOICE') & (F.col('call_direction_cd') == 2) & (F.col('roaming_cd') != 'H'), F.col('usage_amt')/60 )).alias('voice_min_2_R'),
        #F.sum(expr("case when activity_cd = 'VOICE' then usage_amt end "))
        #F.sum(F.when(F.col('activity_cd').isin( ['VOICE_TO_MCC', 'VOICE_MAIL']), F.col('usage_amt')/60)).alias('voice_2_min'),
        F.sum(F.when( (F.col('activity_cd') == 'GPRS') & (F.col('call_direction_cd') == 3) & (F.col('roaming_cd') == 'H'), F.col('usage_amt')/1024/1024)).alias('gprs_mb_3_H'),
        F.sum(F.when( (F.col('activity_cd') == 'GPRS') & (F.col('call_direction_cd') == 3) & (F.col('roaming_cd') != 'H'), F.col('usage_amt')/1024/1024)).alias('gprs_mb_3_R')
    )
    # .join(df_target_sp_CORE, how = 'left', on = 'subscriber_sk')
)

print('num_rows:', df.count())  
# print('num_cols:', len(df.columns))  
df.printSchema()

# num_rows: 374 362 711
# num_rows: 70 742 349
# num_rows: 91 127 187
# 27.10.25: 93 498 382

num_rows: 93498497
root
 |-- transaction_dt: date (nullable = true)
 |-- subscriber_sk: long (nullable = true)
 |-- subscriber_num: string (nullable = true)
 |-- ban_num: long (nullable = true)
 |-- price_plan_cd: string (nullable = true)
 |-- segment_cd: string (nullable = true)
 |-- subs_market_cd: string (nullable = true)
 |-- voice_min_1_H: decimal(38,6) (nullable = true)
 |-- voice_min_2_H: decimal(38,6) (nullable = true)
 |-- voice_min_1_R: decimal(38,6) (nullable = true)
 |-- voice_min_2_R: decimal(38,6) (nullable = true)
 |-- gprs_mb_3_H: decimal(38,11) (nullable = true)
 |-- gprs_mb_3_R: decimal(38,11) (nullable = true)



#### Count NaN

In [6]:
from pyspark.sql.functions import weekofyear

df_count_na = df.withColumn( 'voice_min_1_H_countNA', F.when(F.isnan(F.col('voice_min_1_H')), 1).otherwise(0) ) \
                .withColumn( 'voice_min_2_H_countNA', F.when(F.isnan(F.col('voice_min_2_H')), 1).otherwise(0) ) \
                .withColumn( 'voice_min_1_R_countNA', F.when(F.isnan(F.col('voice_min_1_R')), 1).otherwise(0) ) \
                .withColumn( 'voice_min_2_R_countNA', F.when(F.isnan(F.col('voice_min_2_R')), 1).otherwise(0) ) \
                .withColumn( 'gprs_mb_3_H_countNA', F.when(F.isnan(F.col('gprs_mb_3_H')), 1).otherwise(0) ) \
                .withColumn( 'gprs_mb_3_R_countNA', F.when(F.isnan(F.col('gprs_mb_3_R')), 1).otherwise(0) ) \

# df_count_na = df_count_na.withColumn('week_of_year', weekofyear(df_count_na.transaction_dt))
df_count_na = df_count_na.withColumn('week_of_year', weekofyear(F.col('transaction_dt')))

df_count_na = df_count_na.select('subscriber_sk', 'week_of_year', 
                                 'voice_min_1_H_countNA', 'voice_min_2_H_countNA',
                                 'voice_min_1_R_countNA', 'voice_min_2_R_countNA',
                                 'gprs_mb_3_H_countNA', 'gprs_mb_3_R_countNA'
                                )

df_count_na.printSchema()

root
 |-- subscriber_sk: long (nullable = true)
 |-- week_of_year: integer (nullable = true)
 |-- voice_min_1_H_countNA: integer (nullable = false)
 |-- voice_min_2_H_countNA: integer (nullable = false)
 |-- voice_min_1_R_countNA: integer (nullable = false)
 |-- voice_min_2_R_countNA: integer (nullable = false)
 |-- gprs_mb_3_H_countNA: integer (nullable = false)
 |-- gprs_mb_3_R_countNA: integer (nullable = false)



In [7]:
df_count_na_agg = df_count_na.groupby('subscriber_sk', 'week_of_year') \
                   .agg(F.sum( F.coalesce(F.col('voice_min_1_H_countNA'), F.lit(0.0)) ).alias('voice_min_1_H_countNA'), 
                        F.sum( F.coalesce(F.col('voice_min_2_H_countNA'), F.lit(0.0)) ).alias('voice_min_2_H_countNA'), 
                        F.sum( F.coalesce(F.col('voice_min_1_R_countNA'), F.lit(0.0)) ).alias('voice_min_1_R_countNA'), 
                        F.sum( F.coalesce(F.col('voice_min_2_R_countNA'), F.lit(0.0)) ).alias('voice_min_2_R_countNA'), 
                        F.sum( F.coalesce(F.col('gprs_mb_3_H_countNA'), F.lit(0.0)) ).alias('gprs_mb_3_H_countNA'),
                        F.sum( F.coalesce(F.col('gprs_mb_3_R_countNA'), F.lit(0.0)) ).alias('gprs_mb_3_R_countNA'),
                     )

In [8]:
df_count_na_agg.select('voice_min_1_H_countNA', 'voice_min_2_H_countNA', 'voice_min_1_R_countNA', 'voice_min_2_R_countNA', 'gprs_mb_3_H_countNA', 'gprs_mb_3_R_countNA').dropDuplicates().show(30)

+---------------------+---------------------+---------------------+---------------------+-------------------+-------------------+
|voice_min_1_H_countNA|voice_min_2_H_countNA|voice_min_1_R_countNA|voice_min_2_R_countNA|gprs_mb_3_H_countNA|gprs_mb_3_R_countNA|
+---------------------+---------------------+---------------------+---------------------+-------------------+-------------------+
|                  0.0|                  0.0|                  0.0|                  0.0|                0.0|                0.0|
+---------------------+---------------------+---------------------+---------------------+-------------------+-------------------+



In [9]:
# df_check = spark.read.table('b2b_stg.df_churn_all_pret_mnp_qual_repr_pt') 
# print('num_rows:', df_check.count())
# df_check.show(3)

# # num_rows: 381656880

#### 2. Обращаемость

In [10]:
# select {твои колонки}, n.reason_1, n.reason_2, n.reason_3, n.calls, n.dialog_time
# from {твоя_таблица} a
# left join b2b_stg.mns_churn_pretenzii_all n on a.subscriber_sk = n.subscriber_sk and a.{твоя дата} = n.dt

# n.reason_1, n.reason_2, n.reason_3 – колонки с причинами обращения, при агрегации их можно просто сконкатить в одну строчку
# n.calls – количество обращений в эту дату, при агрегации просто взять sum
# n.dialog_time – время диалога, при агрегации просто взять sum

In [11]:
from pyspark.sql.types import LongType

df_pret = (
            spark.read.table('b2b_stg.mns_churn_pretenzii_all') 
            .withColumnRenamed('ban', 'ban_num')
            .withColumnRenamed('ctn', 'subscriber_num')
            .withColumn('ban_num', F.col('ban_num').cast(LongType()))
            .withColumnRenamed('dt', 'transaction_dt')
          )

 # |-- subscriber_sk: long (nullable = true)
 # |-- subscriber_num: string (nullable = true)
 # |-- ban_num: long (nullable = true)
df_pret.printSchema()

root
 |-- ban_num: long (nullable = true)
 |-- subscriber_num: string (nullable = true)
 |-- subscriber_sk: long (nullable = true)
 |-- transaction_dt: date (nullable = true)
 |-- reason_1: string (nullable = true)
 |-- reason_2: string (nullable = true)
 |-- reason_3: string (nullable = true)
 |-- calls: float (nullable = true)
 |-- dialog_time: float (nullable = true)



In [12]:
df = df.join(df_pret, how = 'left', on = ['subscriber_sk', 'subscriber_num', 'ban_num', 'transaction_dt'])
# df = df.join(df_pret, how = 'left', on = ['subscriber_sk', 'ban_num', 'transaction_dt'])

print('num_rows:', df.count()) 
df.printSchema()
# num_rows: 400 418 864
# num_rows: 400 420 566

# num_rows: 70 742 349
# num_rows: 70 742 620

# num_rows: 91 127 187
# num_rows: 91 127 525
# 27.10.25: 93 498 697

num_rows: 93498812
root
 |-- subscriber_sk: long (nullable = true)
 |-- subscriber_num: string (nullable = true)
 |-- ban_num: long (nullable = true)
 |-- transaction_dt: date (nullable = true)
 |-- price_plan_cd: string (nullable = true)
 |-- segment_cd: string (nullable = true)
 |-- subs_market_cd: string (nullable = true)
 |-- voice_min_1_H: decimal(38,6) (nullable = true)
 |-- voice_min_2_H: decimal(38,6) (nullable = true)
 |-- voice_min_1_R: decimal(38,6) (nullable = true)
 |-- voice_min_2_R: decimal(38,6) (nullable = true)
 |-- gprs_mb_3_H: decimal(38,11) (nullable = true)
 |-- gprs_mb_3_R: decimal(38,11) (nullable = true)
 |-- reason_1: string (nullable = true)
 |-- reason_2: string (nullable = true)
 |-- reason_3: string (nullable = true)
 |-- calls: float (nullable = true)
 |-- dialog_time: float (nullable = true)



In [13]:
400420566 - 400418864

1702

In [14]:
93498697 - 93498382

315

#### 4. Заявки MNP

In [15]:
# -- MNP PortOut Requests:

# start_date = datetime.date(2025, 6, 1) # -- !!!
# end_date = datetime.date(2025, 9, 22) # -- !!!

# start_date = datetime.date(2025, 9, 22) # -- !!!
# end_date = datetime.date(2025, 10, 13) # -- !!!

df_mnp = spark.sql(f"""select subs_key, req_created, count(distinct request_id) req_qnt
                        from biis.dim_crm_mnp_requests_pub r
                        where 1=1
                        and process_type = 'PortOut'
                        and req_created >= '{start_date}'
                        and req_created < '{end_date}'
                        group by subs_key, req_created
                    """)

df_mnp = df_mnp.withColumnRenamed('subs_key', 'subscriber_num') \
               .withColumnRenamed('req_created', 'transaction_dt') 

df_mnp.printSchema()

root
 |-- subscriber_num: string (nullable = true)
 |-- transaction_dt: timestamp (nullable = true)
 |-- req_qnt: long (nullable = false)



In [16]:
df = df.join(df_mnp, how = 'left', on = ['subscriber_num', 'transaction_dt'])

print('num_rows:', df.count()) 
df.printSchema()

# num_rows: 400420566
# num_rows: 70 742 620
# num_rows: 91 127 525
# 27.10.25: 93 498 697

num_rows: 93498812
root
 |-- subscriber_num: string (nullable = true)
 |-- transaction_dt: date (nullable = true)
 |-- subscriber_sk: long (nullable = true)
 |-- ban_num: long (nullable = true)
 |-- price_plan_cd: string (nullable = true)
 |-- segment_cd: string (nullable = true)
 |-- subs_market_cd: string (nullable = true)
 |-- voice_min_1_H: decimal(38,6) (nullable = true)
 |-- voice_min_2_H: decimal(38,6) (nullable = true)
 |-- voice_min_1_R: decimal(38,6) (nullable = true)
 |-- voice_min_2_R: decimal(38,6) (nullable = true)
 |-- gprs_mb_3_H: decimal(38,11) (nullable = true)
 |-- gprs_mb_3_R: decimal(38,11) (nullable = true)
 |-- reason_1: string (nullable = true)
 |-- reason_2: string (nullable = true)
 |-- reason_3: string (nullable = true)
 |-- calls: float (nullable = true)
 |-- dialog_time: float (nullable = true)
 |-- req_qnt: long (nullable = true)



In [17]:
# df.select('req_qnt').dropDuplicates().show()
# df.select('req_qnt').toPandas().value_counts()

#### 5. Качество сети

In [18]:
# Качество связи (тут ban = float):
# select {твои колонки}, m.active_days_data, m.bad_days_data, m.accept_days_data, m.good_days_data
# from {твоя_таблица} a
# left b2b_stg.mns_quality_internet_weekly_without_agg n on a.subscriber_sk = n.subscriber_sk and a.{твоя дата} = TO_TIMESTAMP(m.event_date, 'YYYY-MM-DD HH24:MI:SS') and a.{тсвой бан} = n.ban and a.{номер телефона} =  n.msisdn

In [19]:
from pyspark.sql.types import LongType

df_qual = (
            spark.read.table('b2b_stg.mns_quality_internet_weekly_without_agg') 
            .withColumnRenamed('ban', 'ban_num')
            .withColumnRenamed('msisdn', 'subscriber_num')
            .withColumn('ban_num', F.col('ban_num').cast(LongType()))
            .withColumnRenamed('event_date', 'transaction_dt')
          )

 # |-- subscriber_sk: long (nullable = true)
 # |-- subscriber_num: string (nullable = true)
 # |-- ban_num: long (nullable = true)
df_qual.printSchema()

root
 |-- subscriber_sk: long (nullable = true)
 |-- subscriber_num: string (nullable = true)
 |-- ban_num: long (nullable = true)
 |-- client_inn: string (nullable = true)
 |-- transaction_dt: date (nullable = true)
 |-- active_days_data: integer (nullable = true)
 |-- bad_days_data: integer (nullable = true)
 |-- accept_days_data: integer (nullable = true)
 |-- good_days_data: integer (nullable = true)



In [20]:
df = df.join(df_qual, how = 'left', on = ['subscriber_sk', 'subscriber_num', 'ban_num', 'transaction_dt'])

print('num_rows:', df.count()) 
df.printSchema()

# before: num_rows: 70 742 620
# after: num_rows: 71 451 133

# before: num_rows: 91 127 525
# after: num_rows: 92 023 384
# 27.10.25: 93 498 697

num_rows: 94506341
root
 |-- subscriber_sk: long (nullable = true)
 |-- subscriber_num: string (nullable = true)
 |-- ban_num: long (nullable = true)
 |-- transaction_dt: date (nullable = true)
 |-- price_plan_cd: string (nullable = true)
 |-- segment_cd: string (nullable = true)
 |-- subs_market_cd: string (nullable = true)
 |-- voice_min_1_H: decimal(38,6) (nullable = true)
 |-- voice_min_2_H: decimal(38,6) (nullable = true)
 |-- voice_min_1_R: decimal(38,6) (nullable = true)
 |-- voice_min_2_R: decimal(38,6) (nullable = true)
 |-- gprs_mb_3_H: decimal(38,11) (nullable = true)
 |-- gprs_mb_3_R: decimal(38,11) (nullable = true)
 |-- reason_1: string (nullable = true)
 |-- reason_2: string (nullable = true)
 |-- reason_3: string (nullable = true)
 |-- calls: float (nullable = true)
 |-- dialog_time: float (nullable = true)
 |-- req_qnt: long (nullable = true)
 |-- client_inn: string (nullable = true)
 |-- active_days_data: integer (nullable = true)
 |-- bad_days_data: integer (nullable

In [21]:
71451133 - 70742620

708513

In [22]:
94506341 - 93498697

1007644

#### 7. Попадание в Reprice 8. Изменение TP

In [23]:
# # Сбор 7 и 8:

# res_churn_table = spark.sql('''
#                     select k.*, b.price_plan_type, case when m.repr2q2025b = 1 or m.repr2q2025s = 1 then 1 else 0 end as repr2q2025
#                     from b2b_stg.df_churn_all_pret_mnp_qual k
#                     left join 
#                     (select distinct price_plan_cd, effective_dt, price_plan_type
#                     from b2bba_prod.price_plan_pub) b on k.price_plan_cd = b.price_plan_cd and k.transaction_dt >= b.effective_dt
#                     left join 
#                     (select distinct subs_key, ban_key, subscriber_id, repr2q2025b, repr2q2025s
#                     from b2b_cltv_stg.map_of_base_20250620) m on k.subscriber_num = m.subs_key and k.ban_num = CAST(m.ban_key AS STRING) and k.subscriber_sk = m.subscriber_id
#                     ''')

In [24]:
# resulting_table = 'b2b_stg.df_churn_all_pret_mnp_qual_repr_pt'

# db_name, table_name = resulting_table.split(".")
   
# (
#     res_churn_table
#     .repartition(40)
#     .write.mode("overwrite")
#     .format("parquet")
#     .option("path", f"hdfs://ns-etl/warehouse/tablespace/external/hive/{db_name}.db/{table_name}")
#     .saveAsTable(resulting_table)
# )

In [25]:
# df = spark.read.table('b2b_stg.df_churn_all_pret_mnp_qual_repr_pt') 
        
# print(df.count())
# df.printSchema()

In [26]:
# Для 7, где k – основная таблица:
# res_churn_table = spark.sql('''select k.*, m.repr2q2025
#     from (основная таблица) k
#     left join b2b_stg.mns_churn_reprice m 
#     on k.subscriber_num = m.subs_key and k.ban_num = CAST(m.ban_key AS STRING) and k.subscriber_sk = m.subscriber_id
# ''')

In [27]:
df_repr = ( 
            spark.read.table('b2b_stg.mns_churn_reprice') 
            .withColumnRenamed('subs_key', 'subscriber_num')
            .withColumn('ban_num', F.col('ban_key').cast(StringType()))
            .withColumnRenamed('subscriber_id', 'subscriber_sk')
          )

df = df.join(df_repr, how = 'left', on = ['subscriber_sk', 'subscriber_num', 'ban_num'])

print('num_rows:', df.count()) 
df.printSchema()

# num_rows: 71451133
# num_rows: 92 023 384
# num_rows: 94 506 341

num_rows: 94506341
root
 |-- subscriber_sk: long (nullable = true)
 |-- subscriber_num: string (nullable = true)
 |-- ban_num: long (nullable = true)
 |-- transaction_dt: date (nullable = true)
 |-- price_plan_cd: string (nullable = true)
 |-- segment_cd: string (nullable = true)
 |-- subs_market_cd: string (nullable = true)
 |-- voice_min_1_H: decimal(38,6) (nullable = true)
 |-- voice_min_2_H: decimal(38,6) (nullable = true)
 |-- voice_min_1_R: decimal(38,6) (nullable = true)
 |-- voice_min_2_R: decimal(38,6) (nullable = true)
 |-- gprs_mb_3_H: decimal(38,11) (nullable = true)
 |-- gprs_mb_3_R: decimal(38,11) (nullable = true)
 |-- reason_1: string (nullable = true)
 |-- reason_2: string (nullable = true)
 |-- reason_3: string (nullable = true)
 |-- calls: float (nullable = true)
 |-- dialog_time: float (nullable = true)
 |-- req_qnt: long (nullable = true)
 |-- client_inn: string (nullable = true)
 |-- active_days_data: integer (nullable = true)
 |-- bad_days_data: integer (nullable

In [28]:
# # Для 8, где k – основная таблица:
# select k.*, b.price_plan_type
# from (основная таблица) k
# left join b2b_stg.mns_churn_price_plan b
# on k.price_plan_cd = b.price_plan_cd and k.transaction_dt >= b.effective_dt

In [29]:
df_change_tp = spark.read.table('b2b_stg.mns_churn_price_plan') 

df =  df.alias("k").join(df_change_tp.alias("b"),
                        (F.col("k.price_plan_cd") == F.col("b.price_plan_cd")) &
                        (F.col("k.transaction_dt") >= F.col("b.effective_dt")),
                        how = "left") \
                    .select("k.*", "b.price_plan_type")

print('num_rows:', df.count()) 
df.printSchema()

# num_rows: 71451133
# num_rows: 92023384
# num_rows: 94506341

num_rows: 94506341
root
 |-- subscriber_sk: long (nullable = true)
 |-- subscriber_num: string (nullable = true)
 |-- ban_num: long (nullable = true)
 |-- transaction_dt: date (nullable = true)
 |-- price_plan_cd: string (nullable = true)
 |-- segment_cd: string (nullable = true)
 |-- subs_market_cd: string (nullable = true)
 |-- voice_min_1_H: decimal(38,6) (nullable = true)
 |-- voice_min_2_H: decimal(38,6) (nullable = true)
 |-- voice_min_1_R: decimal(38,6) (nullable = true)
 |-- voice_min_2_R: decimal(38,6) (nullable = true)
 |-- gprs_mb_3_H: decimal(38,11) (nullable = true)
 |-- gprs_mb_3_R: decimal(38,11) (nullable = true)
 |-- reason_1: string (nullable = true)
 |-- reason_2: string (nullable = true)
 |-- reason_3: string (nullable = true)
 |-- calls: float (nullable = true)
 |-- dialog_time: float (nullable = true)
 |-- req_qnt: long (nullable = true)
 |-- client_inn: string (nullable = true)
 |-- active_days_data: integer (nullable = true)
 |-- bad_days_data: integer (nullable

#### Aggregation

In [30]:
from pyspark.sql.functions import weekofyear

df = df.withColumn('week_of_year', weekofyear(F.col('transaction_dt')))

In [31]:
# df_agg = df.groupby('subscriber_sk', 'segment_cd', 'subs_market_cd', 'week_of_year') \
df_agg = df.groupby('subscriber_sk', 'subs_market_cd', 'week_of_year') \
           .agg(F.mean( F.coalesce(F.col('voice_min_1_H'), F.lit(0.0)) ).alias('voice_min_1_H_avg'), 
                F.mean( F.coalesce(F.col('voice_min_2_H'), F.lit(0.0)) ).alias('voice_min_2_H_avg'), 
                F.mean( F.coalesce(F.col('voice_min_1_R'), F.lit(0.0)) ).alias('voice_min_1_R_avg'), 
                F.mean( F.coalesce(F.col('voice_min_2_R'), F.lit(0.0)) ).alias('voice_min_2_R_avg'), 
                F.mean( F.coalesce(F.col('gprs_mb_3_H'), F.lit(0.0)) ).alias('gprs_mb_3_H_avg'),
                F.mean( F.coalesce(F.col('gprs_mb_3_R'), F.lit(0.0)) ).alias('gprs_mb_3_R_avg'),

                # F.concat(F.col('reason_1'), F.col('reason_2'), F.col('reason_3')).alias('pretenzii_reason'),
                F.max('reason_1').alias('reason_1'),
                F.max('reason_2').alias('reason_2'),
                F.max('reason_3').alias('reason_3'),
                
                F.sum( F.coalesce(F.col('calls'), F.lit(0.0)) ).alias('calls_sum'),
                F.sum( F.coalesce(F.col('dialog_time'), F.lit(0.0)) ).alias('dialog_time_sum'),
                F.sum( F.coalesce(F.col('req_qnt'), F.lit(0.0)) ).alias('req_qnt_sum'),
                      
                F.sum( F.coalesce(F.col('active_days_data'), F.lit(0.0)) ).alias('active_days_sum'),
                F.sum( F.coalesce(F.col('bad_days_data'), F.lit(0.0)) ).alias('bad_days_sum'),
                F.sum( F.coalesce(F.col('accept_days_data'), F.lit(0.0)) ).alias('accept_days_sum'),
                F.sum( F.coalesce(F.col('good_days_data'), F.lit(0.0)) ).alias('good_days_sum'),
                      
                F.max('price_plan_type').alias('price_plan_type_max'),
                F.max('repr2q2025').alias('repr2q2025_max')
             )

print('num_rows:', df_agg.count())
df_agg.printSchema()

# num_rows: 71 451 133
# num_rows: 15 225 495
# num_rows: 20 081 473
# num_rows: 19 873 521

num_rows: 19873521
root
 |-- subscriber_sk: long (nullable = true)
 |-- subs_market_cd: string (nullable = true)
 |-- week_of_year: integer (nullable = true)
 |-- voice_min_1_H_avg: double (nullable = true)
 |-- voice_min_2_H_avg: double (nullable = true)
 |-- voice_min_1_R_avg: double (nullable = true)
 |-- voice_min_2_R_avg: double (nullable = true)
 |-- gprs_mb_3_H_avg: double (nullable = true)
 |-- gprs_mb_3_R_avg: double (nullable = true)
 |-- reason_1: string (nullable = true)
 |-- reason_2: string (nullable = true)
 |-- reason_3: string (nullable = true)
 |-- calls_sum: double (nullable = true)
 |-- dialog_time_sum: double (nullable = true)
 |-- req_qnt_sum: double (nullable = true)
 |-- active_days_sum: double (nullable = true)
 |-- bad_days_sum: double (nullable = true)
 |-- accept_days_sum: double (nullable = true)
 |-- good_days_sum: double (nullable = true)
 |-- price_plan_type_max: string (nullable = true)
 |-- repr2q2025_max: integer (nullable = true)



#### Target

In [32]:
print('start_date:', start_date)
print('end_date:', end_date)

start_date: 2025-09-29
end_date: 2025-10-27


In [33]:
from pyspark.sql.functions import weekofyear

# start_date = datetime.date(2025, 6, 1) 
# end_date = datetime.date(2025, 9, 22)

# start_date = datetime.date(2025, 9, 22) # -- !!!
# end_date = datetime.date(2025, 10, 20) # -- !!!

df_churn_target = ( spark.read.table("beemetrics.dm_mobile_subscriber") 
                  .where(F.col("calendar_dt") >= start_date)
                  .where(F.col("calendar_dt") < end_date)
                  .where(F.col("business_type_cd") == "B2B") 
                  .where(F.col("churn_flg") == 1) 
                  .select( F.col('subscriber_sk'), 
                           F.col('calendar_dt').alias('churn_dt'), 
                           F.col('churn_flg')
                         ) )

df_churn_target = df_churn_target.withColumn('week_of_year', weekofyear(df_churn_target.churn_dt))

df_churn_target_agg = df_churn_target.groupby('subscriber_sk', 'week_of_year') \
                                     .agg(F.max('churn_flg').alias('churn_flg'))

# print('num_rows:', df_churn_target_agg.count())

In [34]:
df_agg = df_agg.join(df_churn_target_agg, how = 'left', on = ['subscriber_sk', 'week_of_year'])

print('num_rows:', df_agg.count())
df_agg.printSchema()

# num_rows: 80 000 219
# num_rows: 15 225 495
# num_rows: 20 081 473
# num_rows: 19 873 521

num_rows: 19873521
root
 |-- subscriber_sk: long (nullable = true)
 |-- week_of_year: integer (nullable = true)
 |-- subs_market_cd: string (nullable = true)
 |-- voice_min_1_H_avg: double (nullable = true)
 |-- voice_min_2_H_avg: double (nullable = true)
 |-- voice_min_1_R_avg: double (nullable = true)
 |-- voice_min_2_R_avg: double (nullable = true)
 |-- gprs_mb_3_H_avg: double (nullable = true)
 |-- gprs_mb_3_R_avg: double (nullable = true)
 |-- reason_1: string (nullable = true)
 |-- reason_2: string (nullable = true)
 |-- reason_3: string (nullable = true)
 |-- calls_sum: double (nullable = true)
 |-- dialog_time_sum: double (nullable = true)
 |-- req_qnt_sum: double (nullable = true)
 |-- active_days_sum: double (nullable = true)
 |-- bad_days_sum: double (nullable = true)
 |-- accept_days_sum: double (nullable = true)
 |-- good_days_sum: double (nullable = true)
 |-- price_plan_type_max: string (nullable = true)
 |-- repr2q2025_max: integer (nullable = true)
 |-- churn_flg: inte

In [35]:
df_agg = df_agg.sort(['subscriber_sk', 'week_of_year'])

In [36]:
df_agg = df_agg.join(df_count_na_agg, how = 'left', on = ['subscriber_sk', 'week_of_year'])

print('num_rows:', df_agg.count())
df_agg.printSchema()

# num_rows: 15 225 495
# num_rows: 20 081 473
# num_rows: 19 873 521

num_rows: 19873521
root
 |-- subscriber_sk: long (nullable = true)
 |-- week_of_year: integer (nullable = true)
 |-- subs_market_cd: string (nullable = true)
 |-- voice_min_1_H_avg: double (nullable = true)
 |-- voice_min_2_H_avg: double (nullable = true)
 |-- voice_min_1_R_avg: double (nullable = true)
 |-- voice_min_2_R_avg: double (nullable = true)
 |-- gprs_mb_3_H_avg: double (nullable = true)
 |-- gprs_mb_3_R_avg: double (nullable = true)
 |-- reason_1: string (nullable = true)
 |-- reason_2: string (nullable = true)
 |-- reason_3: string (nullable = true)
 |-- calls_sum: double (nullable = true)
 |-- dialog_time_sum: double (nullable = true)
 |-- req_qnt_sum: double (nullable = true)
 |-- active_days_sum: double (nullable = true)
 |-- bad_days_sum: double (nullable = true)
 |-- accept_days_sum: double (nullable = true)
 |-- good_days_sum: double (nullable = true)
 |-- price_plan_type_max: string (nullable = true)
 |-- repr2q2025_max: integer (nullable = true)
 |-- churn_flg: inte

In [37]:
# resulting_table = 'b2b_stg.df_churn_all_agg_2'
# resulting_table = 'b2b_stg.df_churn_all_agg_131025'
# resulting_table = 'b2b_stg.df_churn_all_agg_201025'
resulting_table = 'b2b_stg.df_churn_all_agg_271025'

db_name, table_name = resulting_table.split(".")
   
(
    df_agg
    .repartition(40)
    .write.mode("overwrite")
    .format("parquet")
    .option("path", f"hdfs://ns-etl/warehouse/tablespace/external/hive/{db_name}.db/{table_name}")
    .saveAsTable(resulting_table)
)

In [38]:
df_check = spark.read.table(resulting_table)

print('num_rows:', df_check.count())
df_check.printSchema()

# num_rows: 15 225 495
# num_rows: 20 081 473
# num_rows: 19 873 521

num_rows: 19873521
root
 |-- subscriber_sk: long (nullable = true)
 |-- week_of_year: integer (nullable = true)
 |-- subs_market_cd: string (nullable = true)
 |-- voice_min_1_H_avg: double (nullable = true)
 |-- voice_min_2_H_avg: double (nullable = true)
 |-- voice_min_1_R_avg: double (nullable = true)
 |-- voice_min_2_R_avg: double (nullable = true)
 |-- gprs_mb_3_H_avg: double (nullable = true)
 |-- gprs_mb_3_R_avg: double (nullable = true)
 |-- reason_1: string (nullable = true)
 |-- reason_2: string (nullable = true)
 |-- reason_3: string (nullable = true)
 |-- calls_sum: double (nullable = true)
 |-- dialog_time_sum: double (nullable = true)
 |-- req_qnt_sum: double (nullable = true)
 |-- active_days_sum: double (nullable = true)
 |-- bad_days_sum: double (nullable = true)
 |-- accept_days_sum: double (nullable = true)
 |-- good_days_sum: double (nullable = true)
 |-- price_plan_type_max: string (nullable = true)
 |-- repr2q2025_max: integer (nullable = true)
 |-- churn_flg: inte